# Tokenization, Stemming, n-gram and skip-gram generation

In this exercise, we'll the emails in the <i>twenty newsgroups</i> dataset to perform simple tokenization, stemming n-gram and skip-gram generation. More information on the <i>Twenty Newsgroups</i> dataset can be found on the [UCI website](http://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.html)

## Setup database connectivity

We'll reuse our module from the previous notebook (***`00_database_connectivity_setup.ipynb`***) to establish connectivity to the database

In [9]:
%run '00_database_connectivity_setup.ipynb'
%matplotlib inline
from IPython.display import display
from IPython.display import HTML

Your connection object is ***`conn`***:
1. Queries: You can run your queries using ***```psql.read_sql("""<YOUR SQL>""", conn)```***.
2. Create/Delete/Updates: You can run these statements using ***```psql.execute("""<YOUR SQL>""", conn)```***, followed by a ***```conn.commit()```*** command to ensure your transaction is committed. Otherwise your changes will be rolledback if you terminate your kernel.

If you created a new connection object (say to connect to a new cluster) as shown in the last section of `00_database_connectivity_setup.ipynb` notebook, use that connection object where needed.

## 1. Create your schema

In [ ]:
sql = """
    create schema YOUR_SCHEMA;
"""
psql.execute(sql, conn)
conn.commit()

## 2. Load the Twenty News Groups dataset into a database table

In [12]:
sql = """
    -- Define external table to fetch data from HDFS
    drop external table if exists YOUR_SCHEMA.twenty_news_groups_ext cascade;
    create external table YOUR_SCHEMA.twenty_news_groups_ext
    (
        doc_id int,
        contents text,
        label text
    ) location ('pxf://hdm1.gphd.local:50070/user/vatsan/dstraining/twenty_news_groups_processed.tsv?profile=hdfstextsimple') 
    format 'CSV' (DELIMITER = E'\t');

    -- create an internal table
    drop table if exists YOUR_SCHEMA.twenty_news_groups cascade;
    create table YOUR_SCHEMA.twenty_news_groups
    as
    (
        select 
            *
        from
            YOUR_SCHEMA.twenty_news_groups_ext
    ) distributed randomly;
"""
psql.execute(sql, conn)
conn.commit()

In [13]:
sql = """
    select 
        *
    from
       YOUR_SCHEMA.twenty_news_groups
    order by random()
    limit 10
"""
df = psql.read_sql(sql, conn)
df.head()
conn.commit()

,doc_id,contents,label
0,7491,From: russotto@eng.umd.edu (Matthew T. Russott...,talk.religion.misc
1,4118,From: wihervaa@messi.uku.fi (Mikko Wihervaara)...,rec.sport.hockey
2,5405,From: chungdan@leland.Stanford.EDU (Zhong Qi I...,sci.med
3,12635,From: mbeaving@bnr.ca (Michael Beavington)\nSu...,rec.motorcycles
4,17799,"From: casper@vxcrna.cern.ch (CASPER,DAVI./PPE)...",talk.politics.mideast


## 3. Tokenize the documents using a simple white-space tokenizer

In [15]:
sql = """
    --1) Create inverted index, compute term frequencies
    drop table if exists YOUR_SCHEMA.twenty_news_groups_term_frequencies cascade;
    create table YOUR_SCHEMA.twenty_news_groups_term_frequencies
    as
    (
        select
            token,
            doc_id,
            count(*) as tf
        from
        (
            select
                doc_id,
                regexp_split_to_table(
                    --convert to lower case
                    --replace newlines/carriage returns to space
                    regexp_replace(lower(contents), E'\\r|\\n', ' ', 'g'), 
                    E'\\\s+'
                ) as token
            from
                YOUR_SCHEMA.twenty_news_groups
        )q
        group by token, doc_id    
    ) distributed randomly;
"""
psql.execute(sql, conn)
conn.commit()
df = psql.read_sql("""select * from YOUR_SCHEMA.twenty_news_groups_term_frequencies limit 10;""", conn)
df.head()

,token,doc_id,tf
0,*,17380,4
1,organization:,12408,1
2,details,14288,1
3,not,7242,1
4,6022(10e20),12229,1


## 4. Remove stopwords 

In [30]:
sql = """
    -- Remove stop words 
    drop table if exists YOUR_SCHEMA.twenty_news_groups_vocabulary cascade;
    create table YOUR_SCHEMA.twenty_news_groups_vocabulary
    as
    (
        select
            token
        from
        (
            select
                token,
                count(distinct doc_id) as num_docs
            from
                YOUR_SCHEMA.twenty_news_groups_term_frequencies
            where
                -- Remove stopwords
                token not in (
                    'a\\'s','able','about','above','according','accordingly','across','actually','after','afterwards','again','against','ain\\'t',
                    'all','allow','allows','almost','alone','along','already','also','although','always','am','among','amongst','an','and',
                    'another','any','anybody','anyhow','anyone','anything','anyway','anyways','anywhere','apart','appear','appreciate',
                    'appropriate','are','aren\\'t','around','as','aside','ask','asking','associated','at','available','away','awfully',
                    'be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','believe','below',
                    'beside','besides','best','better','between','beyond','both','brief','but','by','c\\'mon','c\\'s','came','can','can\\'t',
                    'cannot','cant','cause','causes','certain','certainly','changes','clearly','co','com','come','comes','concerning','consequently',
                    'consider','considering','contain','containing','contains','corresponding','could','couldn\\'t','course','currently','definitely',
                    'described','despite','did','didn\\'t','different','do','does','doesn\\'t','doing','don\\'t','done','down','downwards','during',
                    'each','edu','eg','eight','either','else','elsewhere','enough','entirely','especially','et','etc','even','ever','every',
                    'everybody','everyone','everything','everywhere','ex','exactly','example','except','far','few','fifth','first','five',
                    'followed','following','follows','for','former','formerly','forth','four','from','further','furthermore','get','gets',
                    'getting','given','gives','go','goes','going','gone','got','gotten','greetings','had','hadn\\'t','happens','hardly','has',
                    'hasn\\'t','have','haven\\'t','having','he','he\\'s','hello','help','hence','her','here','here\\'s','hereafter','hereby','herein',
                    'hereupon','hers','herself','hi','him','himself','his','hither','hopefully','how','howbeit','however','i\\'d','i\\'ll','i\\'m',
                    'i\\'ve','ie','if','ignored','immediate','in','inasmuch','inc','indeed','indicate','indicated','indicates','inner','insofar',
                    'instead','into','inward','is','isn\\'t','it','it\\'d','it\\'ll','it\\'s','its','itself','just','keep','keeps','kept','know','knows',
                    'known','last','lately','later','latter','latterly','least','less','lest','let','let\\'s','like','liked','likely','little','look',
                    'looking','looks','ltd','mainly','many','may','maybe','me','mean','meanwhile','merely','might','more','moreover','most','mostly',
                    'much','must','my','myself','name','namely','nd','near','nearly','necessary','need','needs','neither','never','nevertheless','new',
                    'next','nine','no','nobody','non','none','noone','nor','normally','not','nothing','novel','now','nowhere','obviously','of','off',
                    'often','oh','ok','okay','old','on','once','one','ones','only','onto','or','other','others','otherwise','ought','our','ours',
                    'ourselves','out','outside','over','overall','own','particular','particularly','per','perhaps','placed','please','plus','possible',
                    'presumably','probably','provides','que','quite','qv','rather','rd','re','really','reasonably','regarding','regardless','regards',
                    'relatively','respectively','right','said','same','saw','say','saying','says','second','secondly','see','seeing','seem','seemed',
                    'seeming','seems','seen','self','selves','sensible','sent','serious','seriously','seven','several','shall','she','should',
                    'shouldn\\'t','since','six','so','some','somebody','somehow','someone','something','sometime','sometimes','somewhat','somewhere',
                    'soon','sorry','specified','specify','specifying','still','sub','such','sup','sure','t\\'s','take','taken','tell','tends','th',
                    'than','thank','thanks','thanx','that','that\\'s','thats','the','their','theirs','them','themselves','then','thence','there',
                    'there\\'s','thereafter','thereby','therefore','therein','theres','thereupon','these','they','they\\'d','they\\'ll','they\\'re',
                    'they\\'ve','think','third','this','thorough','thoroughly','those','though','three','through','throughout','thru','thus','to',
                    'together','too','took','toward','towards','tried','tries','truly','try','trying','twice','two','un','under','unfortunately',
                    'unless','unlikely','until','unto','up','upon','us','use','used','useful','uses','using','usually','value','various','very',
                    'via','viz','vs','want','wants','was','wasn\\'t','way','we','we\\'d','we\\'ll','we\\'re','we\\'ve','welcome','well','went','were',
                    'weren\\'t','what','what\\'s','whatever','when','whence','whenever','where','where\\'s','whereafter','whereas','whereby','wherein',
                    'whereupon','wherever','whether','which','while','whither','who','who\\'s','whoever','whole','whom','whose','why','will',
                    'willing','wish','with','within','without','won\\'t','wonder','would','would','wouldn\\'t','yes','yet','you','you\\'d','you\\'ll',
                    'you\\'re','you\\'ve','your','yours','yourself','yourselves','zero'
                )
            group by token
        )t1,
        (
            select
                count(distinct doc_id) as corpus_size
            from
                YOUR_SCHEMA.twenty_news_groups_term_frequencies
        ) t2
        where 
            -- Only consider those tokens which have occurred in atleast 2% of the documents
            t1.num_docs >= 0.02*(t2.corpus_size)
    ) distributed randomly;
"""
psql.execute(sql, conn)
conn.commit()

In [40]:
#Print the vocabulary size
sql = """
    select
        count(*) as vocabulary_size
    from
        YOUR_SCHEMA.twenty_news_groups_vocabulary
"""
df = psql.read_sql(sql, conn)
vocabulary_size = df['vocabulary_size'][0]
display(df)
display(psql.read_sql("select * from YOUR_SCHEMA.twenty_news_groups_vocabulary limit 10", conn))
conn.commit()

,vocabulary_size
0,634


,token
0,?
1,heard
2,canada
3,card
4,level
5,data
6,ca
7,place
8,phone
9,friend


634


## 5. Stem the tokens using the PorterStemmer

In many NLP tasks including document classification and information retrieval, it is useful to work with the root form of words in documents than their derived or inflected forms. For instance, 
We could use specialized stemmers in NLP libraries like `NLTK` or `SpaCy` by first installing this libraries on all nodes of the MPP database and invoking them via a PL/Python UDF or we can simply embed a vanilla Python function as described here: [Porter Stemmer (Python)](http://tartarus.org/martin/PorterStemmer/python.txt), directly into a UDF for the sake of demonstrating its usage.

In [18]:
sql = """
    drop function if exists YOUR_SCHEMA.stem_tokens(text[]);
    create function YOUR_SCHEMA.stem_tokens(
        tokens text[]
    )
    returns setof text
    as
    $$
        '''Porter Stemming Algorithm
        This is the Porter stemming algorithm, ported to Python from the
        version coded up in ANSI C by the author. It may be be regarded
        as canonical, in that it follows the algorithm presented in

        Porter, 1980, An algorithm for suffix stripping, Program, Vol. 14,
        no. 3, pp 130-137,

        only differing from it at the points maked --DEPARTURE-- below.

        See also http://www.tartarus.org/~martin/PorterStemmer

        The algorithm as described in the paper could be exactly replicated
        by adjusting the points of DEPARTURE, but this is barely necessary,
        because (a) the points of DEPARTURE are definitely improvements, and
        (b) no encoding of the Porter stemmer I have seen is anything like
        as exact as this version, even with the points of DEPARTURE!

        Vivake Gupta (v@nano.com)

        Release 1: January 2001

        Further adjustments by Santiago Bruno (bananabruno@gmail.com)
        to allow word input not restricted to one word per line, leading
        to:

        release 2: July 2008
        '''

        import sys

        class PorterStemmer:

            def __init__(self):
                '''The main part of the stemming algorithm starts here.
                b is a buffer holding a word to be stemmed. The letters are in b[k0],
                b[k0+1] ... ending at b[k]. In fact k0 = 0 in this demo program. k is
                readjusted downwards as the stemming progresses. Zero termination is
                not in fact used in the algorithm.

                Note that only lower case sequences are stemmed. Forcing to lower case
                should be done before stem(...) is called.
                '''

                self.b = ""  # buffer for word to be stemmed
                self.k = 0
                self.k0 = 0
                self.j = 0   # j is a general offset into the string

            def cons(self, i):
                '''cons(i) is TRUE <=> b[i] is a consonant.'''
                if self.b[i] == 'a' or self.b[i] == 'e' or self.b[i] == 'i' or self.b[i] == 'o' or self.b[i] == 'u':
                    return 0
                if self.b[i] == 'y':
                    if i == self.k0:
                        return 1
                    else:
                        return (not self.cons(i - 1))
                return 1

            def m(self):
                '''m() measures the number of consonant sequences between k0 and j.
                if c is a consonant sequence and v a vowel sequence, and <..>
                indicates arbitrary presence,

                   <c><v>       gives 0
                   <c>vc<v>     gives 1
                   <c>vcvc<v>   gives 2
                   <c>vcvcvc<v> gives 3
                   ....
                '''
                n = 0
                i = self.k0
                while 1:
                    if i > self.j:
                        return n
                    if not self.cons(i):
                        break
                    i = i + 1
                i = i + 1
                while 1:
                    while 1:
                        if i > self.j:
                            return n
                        if self.cons(i):
                            break
                        i = i + 1
                    i = i + 1
                    n = n + 1
                    while 1:
                        if i > self.j:
                            return n
                        if not self.cons(i):
                            break
                        i = i + 1
                    i = i + 1

            def vowelinstem(self):
                '''vowelinstem() is TRUE <=> k0,...j contains a vowel'''
                for i in range(self.k0, self.j + 1):
                    if not self.cons(i):
                        return 1
                return 0

            def doublec(self, j):
                '''doublec(j) is TRUE <=> j,(j-1) contain a double consonant.'''
                if j < (self.k0 + 1):
                    return 0
                if (self.b[j] != self.b[j-1]):
                    return 0
                return self.cons(j)

            def cvc(self, i):
                '''cvc(i) is TRUE <=> i-2,i-1,i has the form consonant - vowel - consonant
                and also if the second c is not w,x or y. this is used when trying to
                restore an e at the end of a short  e.g.

                   cav(e), lov(e), hop(e), crim(e), but
                   snow, box, tray.
                '''
                if i < (self.k0 + 2) or not self.cons(i) or self.cons(i-1) or not self.cons(i-2):
                    return 0
                ch = self.b[i]
                if ch == 'w' or ch == 'x' or ch == 'y':
                    return 0
                return 1

            def ends(self, s):
                '''ends(s) is TRUE <=> k0,...k ends with the string s.'''
                length = len(s)
                if s[length - 1] != self.b[self.k]: # tiny speed-up
                    return 0
                if length > (self.k - self.k0 + 1):
                    return 0
                if self.b[self.k-length+1:self.k+1] != s:
                    return 0
                self.j = self.k - length
                return 1

            def setto(self, s):
                '''setto(s) sets (j+1),...k to the characters in the string s, readjusting k.'''
                length = len(s)
                self.b = self.b[:self.j+1] + s + self.b[self.j+length+1:]
                self.k = self.j + length

            def r(self, s):
                '''r(s) is used further down.'''
                if self.m() > 0:
                    self.setto(s)

            def step1ab(self):
                '''step1ab() gets rid of plurals and -ed or -ing. e.g.

                   caresses  ->  caress
                   ponies    ->  poni
                   ties      ->  ti
                   caress    ->  caress
                   cats      ->  cat

                   feed      ->  feed
                   agreed    ->  agree
                   disabled  ->  disable

                   matting   ->  mat
                   mating    ->  mate
                   meeting   ->  meet
                   milling   ->  mill
                   messing   ->  mess

                   meetings  ->  meet
                '''
                if self.b[self.k] == 's':
                    if self.ends("sses"):
                        self.k = self.k - 2
                    elif self.ends("ies"):
                        self.setto("i")
                    elif self.b[self.k - 1] != 's':
                        self.k = self.k - 1
                if self.ends("eed"):
                    if self.m() > 0:
                        self.k = self.k - 1
                elif (self.ends("ed") or self.ends("ing")) and self.vowelinstem():
                    self.k = self.j
                    if self.ends("at"):   self.setto("ate")
                    elif self.ends("bl"): self.setto("ble")
                    elif self.ends("iz"): self.setto("ize")
                    elif self.doublec(self.k):
                        self.k = self.k - 1
                        ch = self.b[self.k]
                        if ch == 'l' or ch == 's' or ch == 'z':
                            self.k = self.k + 1
                    elif (self.m() == 1 and self.cvc(self.k)):
                        self.setto("e")

            def step1c(self):
                '''step1c() turns terminal y to i when there is another vowel in the stem.'''
                if (self.ends("y") and self.vowelinstem()):
                    self.b = self.b[:self.k] + 'i' + self.b[self.k+1:]

            def step2(self):
                '''step2() maps double suffices to single ones.
                so -ization ( = -ize plus -ation) maps to -ize etc. note that the
                string before the suffix must give m() > 0.
                '''
                if self.b[self.k - 1] == 'a':
                    if self.ends("ational"):   self.r("ate")
                    elif self.ends("tional"):  self.r("tion")
                elif self.b[self.k - 1] == 'c':
                    if self.ends("enci"):      self.r("ence")
                    elif self.ends("anci"):    self.r("ance")
                elif self.b[self.k - 1] == 'e':
                    if self.ends("izer"):      self.r("ize")
                elif self.b[self.k - 1] == 'l':
                    if self.ends("bli"):       self.r("ble") # --DEPARTURE--
                    # To match the published algorithm, replace this phrase with
                    #   if self.ends("abli"):      self.r("able")
                    elif self.ends("alli"):    self.r("al")
                    elif self.ends("entli"):   self.r("ent")
                    elif self.ends("eli"):     self.r("e")
                    elif self.ends("ousli"):   self.r("ous")
                elif self.b[self.k - 1] == 'o':
                    if self.ends("ization"):   self.r("ize")
                    elif self.ends("ation"):   self.r("ate")
                    elif self.ends("ator"):    self.r("ate")
                elif self.b[self.k - 1] == 's':
                    if self.ends("alism"):     self.r("al")
                    elif self.ends("iveness"): self.r("ive")
                    elif self.ends("fulness"): self.r("ful")
                    elif self.ends("ousness"): self.r("ous")
                elif self.b[self.k - 1] == 't':
                    if self.ends("aliti"):     self.r("al")
                    elif self.ends("iviti"):   self.r("ive")
                    elif self.ends("biliti"):  self.r("ble")
                elif self.b[self.k - 1] == 'g': # --DEPARTURE--
                    if self.ends("logi"):      self.r("log")
                # To match the published algorithm, delete this phrase

            def step3(self):
                '''step3() dels with -ic-, -full, -ness etc. similar strategy to step2.'''
                if self.b[self.k] == 'e':
                    if self.ends("icate"):     self.r("ic")
                    elif self.ends("ative"):   self.r("")
                    elif self.ends("alize"):   self.r("al")
                elif self.b[self.k] == 'i':
                    if self.ends("iciti"):     self.r("ic")
                elif self.b[self.k] == 'l':
                    if self.ends("ical"):      self.r("ic")
                    elif self.ends("ful"):     self.r("")
                elif self.b[self.k] == 's':
                    if self.ends("ness"):      self.r("")

            def step4(self):
                '''step4() takes off -ant, -ence etc., in context <c>vcvc<v>.'''
                if self.b[self.k - 1] == 'a':
                    if self.ends("al"): pass
                    else: return
                elif self.b[self.k - 1] == 'c':
                    if self.ends("ance"): pass
                    elif self.ends("ence"): pass
                    else: return
                elif self.b[self.k - 1] == 'e':
                    if self.ends("er"): pass
                    else: return
                elif self.b[self.k - 1] == 'i':
                    if self.ends("ic"): pass
                    else: return
                elif self.b[self.k - 1] == 'l':
                    if self.ends("able"): pass
                    elif self.ends("ible"): pass
                    else: return
                elif self.b[self.k - 1] == 'n':
                    if self.ends("ant"): pass
                    elif self.ends("ement"): pass
                    elif self.ends("ment"): pass
                    elif self.ends("ent"): pass
                    else: return
                elif self.b[self.k - 1] == 'o':
                    if self.ends("ion") and (self.b[self.j] == 's' or self.b[self.j] == 't'): pass
                    elif self.ends("ou"): pass
                    # takes care of -ous
                    else: return
                elif self.b[self.k - 1] == 's':
                    if self.ends("ism"): pass
                    else: return
                elif self.b[self.k - 1] == 't':
                    if self.ends("ate"): pass
                    elif self.ends("iti"): pass
                    else: return
                elif self.b[self.k - 1] == 'u':
                    if self.ends("ous"): pass
                    else: return
                elif self.b[self.k - 1] == 'v':
                    if self.ends("ive"): pass
                    else: return
                elif self.b[self.k - 1] == 'z':
                    if self.ends("ize"): pass
                    else: return
                else:
                    return
                if self.m() > 1:
                    self.k = self.j

            def step5(self):
                '''step5() removes a final -e if m() > 1, and changes -ll to -l if
                m() > 1.
                '''
                self.j = self.k
                if self.b[self.k] == 'e':
                    a = self.m()
                    if a > 1 or (a == 1 and not self.cvc(self.k-1)):
                        self.k = self.k - 1
                if self.b[self.k] == 'l' and self.doublec(self.k) and self.m() > 1:
                    self.k = self.k -1

            def stem(self, p, i, j):
                '''In stem(p,i,j), p is a char pointer, and the string to be stemmed
                is from p[i] to p[j] inclusive. Typically i is zero and j is the
                offset to the last character of a string, (p[j+1] == '\\0'). The
                stemmer adjusts the characters p[i] ... p[j] and returns the new
                end-point of the string, k. Stemming never increases word length, so
                i <= k <= j. To turn the stemmer into a module, declare 'stem' as
                extern, and delete the remainder of this file.
                '''
                # copy the parameters into statics
                self.b = p
                self.k = j
                self.k0 = i
                if self.k <= self.k0 + 1:
                    return self.b # --DEPARTURE--

                # With this line, strings of length 1 or 2 don't go through the
                # stemming process, although no mention is made of this in the
                # published algorithm. Remove the line to match the published
                # algorithm.

                self.step1ab()
                self.step1c()
                self.step2()
                self.step3()
                self.step4()
                self.step5()
                return self.b[self.k0:self.k+1] 
        #Stem the tokens
        p = PorterStemmer()
        results = []
        for tok in tokens:
            results.append(p.stem(tok, 0, len(tok)-1))
        return results
    $$ language plpythonu;
"""
psql.execute(sql, conn)
conn.commit()

In [19]:
sql = """
    select
        YOUR_SCHEMA.stem_tokens(tokens)
    from
    (
        select
            regexp_split_to_array(
                'Running is good for your health, as is swimming',
                E'\\\s+'
            ) as tokens
    )q
"""
df = psql.read_sql(sql, conn)
display(df)

,stem_tokens
0,Run
1,is
2,good
3,for
4,your
5,"health,"
6,as
7,is
8,swim


## 6. Generate n-grams and skip-grams 

Skip grams are a generalization of n-grams where we relax the constraint that the tokens should be adjacent to each other. For instance, let's consider the phrase `"insurgents killed in ongoing fighting"`, if we tokenize this phrase and were to extract <i>unigrams</i> from it, we'd get `['insurgents', 'killed', 'in', 'ongoing', 'fighting']`. If we were to extract <i>bigrams</i>, then we'd get `[('insurgents','killed'), ('killed','in'), ('in','ongoing'), ('ongoing', 'fighting')]`. If we were to generalize this to `k-skip-n-gram`, for instance say `2-skip-bigram` we'd get the following tokens: 

`[('insurgents', 'killed'), 
    ('insurgents', 'in'), 
    ('insurgents', 'ongoing'), 
    ('killed', 'in'), 
    ('killed', 'ongoing'), 
    ('killed', 'fighting'), 
    ('in', 'ongoing'), 
    ('in', 'fighting'), 
    ('ongoing', 'fighting')]`
    
In many NLP tasks, skip-grams have achieved comparable accuracy to models trained on n-grams with far fewer training samples. For more information on skip-grams please refer to [A Closer Look at Skip-gram Modelling](http://homepages.inf.ed.ac.uk/ballison/pdf/lrec_skipgrams.pdf)

### Define the UDF to generate k-skip-n-grams

In [13]:
sql = """
    drop function if exists YOUR_SCHEMA.generate_k_skip_n_grams(text[], int, int) cascade;
    create function YOUR_SCHEMA.generate_k_skip_n_grams(
        tokens text[],
        k int,
        n int
    )
    returns setof text[]
    as
    $$
        from itertools import combinations
        def is_valid_k_skip(lst, k):
            '''
                Validate if a given n-gram list (of token indices) contains valid k-skips.
                For instance, a 2-skip bigram, includes 0-skip bigrams, 1-skip bigrams 
                and 2-skip bigrams.
            '''
            idx_diff = [(next_idx - current_idx) for current_idx, next_idx in zip(lst, lst[1:])]
            valid_skips = set(range(k+2))
            return len(set(idx_diff))==1 and (set(idx_diff).pop() in valid_skips)

        def generate_k_skip_n_gram(lst, k, n):
            '''
                Return all k-skip, n-grams as defined in 
                http://homepages.inf.ed.ac.uk/ballison/pdf/lrec_skipgrams.pdf
                ex: "Insurgents killed in ongoing fighting"
                Bi-grams = {insurgents killed, killed in, in ongoing,
                ongoing fighting}.
                2-skip-bi-grams = {insurgents killed, insurgents in,
                insurgents ongoing, killed in, killed ongoing, killed
                fighting, in ongoing, in fighting, ongoing fighting}
                Tri-grams = {insurgents killed in, killed in ongoing, in
                ongoing fighting}.
                2-skip-tri-grams = {insurgents killed in, insurgents killed
                ongoing, insurgents killed fighting, insurgents in ongoing,
                insurgents in fighting, insurgents ongoing fighting, killed
                in ongoing, killed in fighting, killed ongoing fighting, in
                ongoing fighting}.
            '''
            if n > len(lst) or k > len(lst):
                raise 'Invalid values for n:{0} or k:{1}'.format(n, k) 
            #Optimization for normal n-grams (0-skip-n-grams)
            if(k==0):
                return zip(*[lst[i:] for i in range(n)])
            else:
                idx_token_dict = dict(map(lambda el: reversed(el), enumerate(lst)))
                n_grams = combinations(lst, n)
                return filter(lambda ngram: is_valid_k_skip(map(lambda el: idx_token_dict[el], ngram), k), n_grams)
        return generate_k_skip_n_gram(tokens, k, n)
    $$language plpythonu;
"""
psql.execute(sql, conn)
conn.commit()

### Invoke the UDF to generate bigrams and trigrams on a sample list of emails from the <Twenty News Groups> table

In [11]:
sql = """
    select
        doc_id,
        contents,
        YOUR_SCHEMA.generate_k_skip_n_grams(
            tokens,
            0, --no. of skips (k)
            2  --n-gram (n=2 => bigram, n=3 => trigram etc.) 
        ) as bigram
    from
    (
        select
            doc_id,
            contents,
            regexp_split_to_array(
                -- replace non-alpha characters, except spaces
                regexp_replace(
                    --convert to lower case
                    --replace newlines/carriage returns to space
                    regexp_replace(lower(contents), E'\\r|\\n', ' ', 'g'),
                    E'[^a-zA-Z0-9_ ]', '', 'g'
                ),
                E'\\\s+'
            ) as tokens
        from
            YOUR_SCHEMA.twenty_news_groups
        limit 1
    )q;
"""
df = psql.read_sql(sql, conn)
HTML(df.to_html())
conn.commit()

,doc_id,contents,bigram
0,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[from, kmr4pocwruedu]"
1,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[kmr4pocwruedu, keith]"
2,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[keith, m]"
3,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[m, ryan]"
4,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[ryan, subject]"
5,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[subject, re]"
6,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[re, you]"
7,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[you, will]"
8,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[will, all]"
9,60,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,"[all, go]"


### Invoke the UDF to generate `2-skip-bi-grams`

In [14]:
sql = """
    select
        doc_id,
        contents,
        YOUR_SCHEMA.generate_k_skip_n_grams(
            tokens,
            2, --no. of skips (k)
            2  --n-gram (n=2 => bigram, n=3 => trigram etc.) 
        ) as bigram
    from
    (
        select
            doc_id,
            contents,
            regexp_split_to_array(
                -- replace non-alpha characters, except spaces
                regexp_replace(
                    --convert to lower case
                    --replace newlines/carriage returns to space
                    regexp_replace(lower(contents), E'\\r|\\n', ' ', 'g'),
                    E'[^a-zA-Z0-9_ ]', '', 'g'
                ),
                E'\\\s+'
            ) as tokens
        from
            YOUR_SCHEMA.twenty_news_groups
        limit 1
    )q;
"""
df = psql.read_sql(sql, conn)
HTML(df.to_html())
conn.commit()

,doc_id,contents,bigram
0,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[from, liveseysolntzewpdsgicom]"
1,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[from, livesey]"
2,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[liveseysolntzewpdsgicom, livesey]"
3,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[liveseysolntzewpdsgicom, subject]"
4,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[jon, jon]"
5,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[jon, ]"
6,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[livesey, subject]"
7,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[livesey, yet]"
8,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[subject, yet]"
9,73,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,"[subject, more]"
